In [14]:
from scipy.stats import t
import pandas as pd
import seaborn as sns
import numpy as np
import plotly.express as px
from scipy.stats import norm
pd.set_option('display.float_format', lambda x: '{:.2f}'.format(x))
from typing import List
import matplotlib.pyplot as plt
import math
from scipy.stats import ttest_ind
from statsmodels.stats.weightstats import DescrStatsW, CompareMeans


In [ ]:
def obter_intervalo_confianca(dataframe: pd.DataFrame):    
    conf_level = 0.95
    media = dataframe['preco'].mean()
    std = dataframe['preco'].std()
    n = dataframe.shape[0]
    grausLiberdade = n - 1
    t_value = t.ppf((1 + conf_level) / 2, grausLiberdade)
    inf = media - t_value * std / np.sqrt(n)
    sup = media + t_value * std / np.sqrt(n)
    print('Média:', round(media), ' Desvio padrão:', round(std))
    print('Intervalo de confiança:', round(inf), '-', round(sup))

In [2]:
def obter_resumo(tipo_imovel: str, bairro: str = None, banheiro: int = None, quartos: int = None) -> pd.DataFrame:
    query = f'  tipo_imovel == "{tipo_imovel}"'
    if bairro is not None:
        query += f' and bairro_teste == "{bairro}"  '
    if banheiro is not None:
        query += f' and banheiro == {banheiro} '
    if quartos is not None:
        query += f' and quarto == {quartos} '
    base_original = pd.read_parquet('../data/processed/base_casa.parquet')
    base_original = base_original.query(query)
    return base_original

In [3]:
def obter_estatistica(bairros: List[str], base_original_completa: pd.DataFrame) -> pd.DataFrame:
    lista_dados = []
    for bairro in bairros:
        dados_temp = base_original_completa.query(
            f'bairro_teste == "{bairro}"')
        lista_dados.append(
            {
                'bairro': bairro,
                'media_preco':  round(dados_temp['preco'].mean(), 2),
                'mediana_preco':  round(dados_temp['preco'].median(), 2),
                'moda': round(dados_temp['preco'].mode()[0], 2),
                'desvio_padrao': round(dados_temp['preco'].std(), 2),
                'total_imoveis': dados_temp.shape[0],
                'coeficiente_variacao': (round(dados_temp['preco'].std(), 2) / round(dados_temp['preco'].mean(), 2)) * 100
            }
        )
    df = pd.DataFrame(lista_dados)
    return df
  

In [6]:
base_original = obter_resumo(tipo_imovel='Apartamento')
base_original['quarto'] = base_original['quarto'].str.replace('1 Quarto', '1').astype('int32')
base_original['garagem'] = base_original['garagem'].str.replace('--', '0').astype('int32')
base_original['metragem'] = base_original['metragem'].astype('int32')
base_original[['banheiro', 'garagem']] = base_original[['banheiro', 'garagem']].astype('int32')
base_original

,tipo_imovel,nome,preco,metragem,quarto,banheiro,garagem,ID_CASA,bairro_teste
0,Apartamento,"Apartamento com 2 Quartos à Venda, 65m²",310000.00,65,2,2,1,2699716579,Jardim Botânico
2,Apartamento,"Apartamento com 2 Quartos à Venda, 71m²",403500.00,71,2,2,2,2690493538,Bonfim Paulista
3,Apartamento,"Apartamento com 2 Quartos à Venda, 54m²",290000.00,54,2,2,1,2667652105,Nova Aliança
4,Apartamento,"Apartamento com 2 Quartos à Venda, 45m²",166420.00,45,2,1,1,2682840782,Conjunto Habitacional Jardim Das Palmeiras
5,Apartamento,"Apartamento com 2 Quartos à Venda, 64m²",370000.00,64,2,2,2,2688027260,Vila Ana Maria
...,...,...,...,...,...,...,...,...,...
9967,Apartamento,"Apartamento com 3 Quartos à Venda, 81m²",540000.00,81,3,2,2,2696267182,Vila Do Golf
9968,Apartamento,"Apartamento com Quarto à Venda, 61m²",280000.00,61,1,1,1,2476716299,Centro
9969,Apartamento,"Apartamento com 3 Quartos à Venda, 83m²",375000.00,83,3,3,2,2693112449,Jardim São Luiz
9970,Apartamento,"Apartamento com 2 Quartos à Venda, 48m²",175000.00,48,2,1,1,2572455642,Lagoinha


In [5]:
base_original = base_original.drop_duplicates()
base_original.shape

(8395, 9)

- Total Imoveis

In [7]:
base_original.groupby('bairro_teste').size().reset_index(name='count').sort_values(by='count', ascending=False).nlargest(50, 'count')


,bairro_teste,count
9,Centro,872
34,Jardim Botânico,871
98,Nova Aliança,805
51,Jardim Irajá,414
74,Jardim Paulista,388
137,Ribeirânia,273
73,Jardim Palma Travassos,218
8,Campos Eliseos,208
69,Jardim Olhos D Agua,202
7,Bosque Das Juritis,201


In [8]:
bairros = [
    'Jardim Botânico',
    'Centro',
    'Nova Aliança',
    'Jardim Irajá',
    'Sumarezinho', 
    'Vila Monte Alegre', 
    'Bonfim Paulista',
    'Ribeirânia',
    'Campos Eliseos'
]
base_bairros = base_original[base_original['bairro_teste'].isin(bairros)]

In [9]:
base_bairros.pivot_table(index='garagem', columns='bairro_teste', aggfunc='size', fill_value=0)

bairro_teste,Bonfim Paulista,Campos Eliseos,Centro,Jardim Botânico,Jardim Irajá,Nova Aliança,Ribeirânia,Sumarezinho,Vila Monte Alegre
garagem,,,,,,,,,
0,3,10,112,12,8,11,0,3,0
1,90,174,517,262,180,387,227,130,75
2,54,24,197,468,161,356,32,5,9
3,10,0,31,81,41,46,0,0,2
4,5,0,15,35,21,5,14,1,0
5,0,0,0,5,2,0,0,0,0
6,2,0,0,7,1,0,0,0,0
8,1,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,1


In [10]:
base_bairros.pivot_table(index='quarto', columns='bairro_teste', aggfunc='size', fill_value=0)

bairro_teste,Bonfim Paulista,Campos Eliseos,Centro,Jardim Botânico,Jardim Irajá,Nova Aliança,Ribeirânia,Sumarezinho,Vila Monte Alegre
quarto,,,,,,,,,
1,8,4,124,129,41,228,74,0,22
2,110,89,196,294,97,287,165,123,50
3,44,113,473,394,240,276,30,14,14
4,1,1,77,54,35,14,4,2,0
5,3,1,2,0,1,0,0,0,0
20,0,0,0,0,0,0,0,0,1


In [11]:
base_banheiro = base_bairros.pivot_table(index='banheiro', columns='bairro_teste', aggfunc='size', fill_value=0).sort_index().reset_index()
base_banheiro

bairro_teste,banheiro,Bonfim Paulista,Campos Eliseos,Centro,Jardim Botânico,Jardim Irajá,Nova Aliança,Ribeirânia,Sumarezinho,Vila Monte Alegre
0,1,80,109,202,179,73,301,156,83,44
1,2,40,65,247,292,149,195,89,55,41
2,3,12,33,264,132,90,172,9,1,1
3,4,22,0,88,103,50,69,6,0,0
4,5,7,1,61,141,41,61,8,0,0
5,6,3,0,6,15,9,3,4,0,0
6,7,0,0,2,5,1,3,1,0,0
7,8,2,0,0,0,0,1,0,0,0
8,9,0,0,2,2,1,0,0,0,0
9,12,0,0,0,1,0,0,0,0,0


# intervalo de confiança

In [12]:

base_bairros['bairro_teste'] = base_bairros['bairro_teste'].str.strip()
dados_estatisticos = []
for bairro in bairros:
    amostras = {}
    base_temp = base_bairros.query(f'bairro_teste == "{bairro}"')
    amostras['bairro'] = bairro
    amostras['total_imoveis'] = base_temp.shape[0]
    amostras['media'] = round(base_temp["preco"].mean(), 2)
    amostras['max_preco'] = round(base_temp["preco"].max(), 2)
    amostras['min_preco'] = round(base_temp["preco"].min(), 2)
    amostras['mediana'] = round(base_temp["preco"].median(), 2)
    amostras['moda'] = round(base_temp["preco"].mode()[0], 2)
    amostras['desvio_padrao'] = round(base_temp["preco"].std(), 2)
    amostras['erro_padrao'] = round(base_temp["preco"].std(), 2) / math.sqrt(base_temp.shape[0])
    amostras['graus_liberdade'] = base_temp.shape[0] - 1
    amostras['ic_limite_inferior'] = norm.interval(0.95,  loc=round(base_temp["preco"].mean(), 2), scale=round(base_temp["preco"].std(), 2) / math.sqrt(base_temp.shape[0]))[0]
    amostras['ic_limite_superior'] = norm.interval(0.95,  loc=round(base_temp["preco"].mean(), 2), scale=round(base_temp["preco"].std(), 2) / math.sqrt(base_temp.shape[0]))[1]
    dados_estatisticos.append(amostras)
base_dados_bairros_selecionados = pd.DataFrame(dados_estatisticos)
base_dados_bairros_selecionados


/tmp/ipykernel_46930/2173231052.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_bairros['bairro_teste'] = base_bairros['bairro_teste'].str.strip()


,bairro,total_imoveis,media,max_preco,min_preco,mediana,moda,desvio_padrao,erro_padrao,graus_liberdade,ic_limite_inferior,ic_limite_superior
0,Jardim Botânico,871,743104.02,8000000.00,180000.00,550000.00,380000.00,730386.00,24748.19,870,694598.47,791609.57
1,Centro,872,395621.41,3100000.00,92000.00,350500.50,450000.00,217811.05,7376.01,871,381164.69,410078.13
2,Nova Aliança,805,519445.03,2290000.00,133000.00,430000.00,350000.00,297569.61,10487.95,804,498889.02,540001.04
3,Jardim Irajá,414,606743.81,2150000.00,150000.00,450000.00,430000.00,365358.75,17956.40,413,571549.91,641937.71
4,Sumarezinho,139,243888.56,590000.00,117000.00,230000.00,170000.00,70137.18,5948.96,138,232228.82,255548.30
5,Vila Monte Alegre,87,277466.68,2500000.00,125000.00,250000.00,185000.00,252794.05,27102.37,86,224347.02,330586.34
6,Bonfim Paulista,166,598812.09,7451000.00,104652.00,297000.00,190000.00,1003035.61,77850.66,165,446227.60,751396.58
7,Ribeirânia,273,432458.45,2301000.00,120000.00,350000.00,150000.00,401668.14,24310.06,272,384811.60,480105.30
8,Campos Eliseos,208,272279.33,636000.00,110000.00,240000.00,210000.00,103170.67,7153.60,207,258258.53,286300.13


# Teste de hipóteses

- Verificar se os preços dos apartamentos de até 60 m2 são maiores que os apartamentos  de 60 metros

In [13]:
significancia = 0.05
confianca = 1 - significancia

In [15]:
bairros = [
    ('Jardim Botânico', 100), 
    ('Centro', 80), 
    ('Nova Aliança', 100), 
    ('Jardim Irajá', 35), 
    ('Sumarezinho', 50), 
    ('Vila Monte Alegre', 33), 
    ('Bonfim Paulista', 60),
    ('Ribeirânia', 60),
    ('Campos Eliseos', 80)
    
]
metro = 60
for bairro in bairros:
    try:
        print('*' * 20, bairro[0], '*' * 20)
        base_temp = base_bairros.query(f'bairro_teste == "{ bairro[0]}"')
        print(base_temp.shape)
        base_menor_area = base_temp.query(f'metragem < {metro}').sample(n=bairro[1], random_state=101).preco
        base_maior_area = base_temp.query(f'metragem >= {metro}').sample(n=bairro[1], random_state=101).preco
        test_menor_area = DescrStatsW(base_menor_area)
        test_maior_area = DescrStatsW(base_maior_area)
        test_preco = test_menor_area.get_compare(test_maior_area)
        z, p_value = test_preco.ztest_ind(alternative='larger', value=0)
        if p_value < significancia:
            print(f'Rejeitamos a hipótese nula: os preços dos apartamentos de até {metro} m² são significativamente maiores.')
        else:
            print(f'Não rejeitamos a hipótese nula: não há evidências suficientes para afirmar que os preços dos apartamentos de até {metro} m² são maiores.')
    except Exception as e:
        print(bairro)
    print()


******************** Jardim Botânico ********************
(871, 9)
Não rejeitamos a hipótese nula: não há evidências suficientes para afirmar que os preços dos apartamentos de até 60 m² são maiores.

******************** Centro ********************
(872, 9)
Não rejeitamos a hipótese nula: não há evidências suficientes para afirmar que os preços dos apartamentos de até 60 m² são maiores.

******************** Nova Aliança ********************
(805, 9)
Não rejeitamos a hipótese nula: não há evidências suficientes para afirmar que os preços dos apartamentos de até 60 m² são maiores.

******************** Jardim Irajá ********************
(414, 9)
Não rejeitamos a hipótese nula: não há evidências suficientes para afirmar que os preços dos apartamentos de até 60 m² são maiores.

******************** Sumarezinho ********************
(139, 9)
Não rejeitamos a hipótese nula: não há evidências suficientes para afirmar que os preços dos apartamentos de até 60 m² são maiores.

*******************